In [85]:
import requests
from IPython.display import HTML
import csv
import dateutil.parser
from newspaper import Article
import pandas
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.utils import shuffle

In [86]:
df = pandas.read_csv('bingNewsMatch.csv')
df = shuffle(df) 
training_limit = math.ceil(len(df) * 0.8)
validation_limit = len(df)- training_limit
i=0
training_data=[]
validation_data=[]
for index, row in df.iterrows():
    date_pub = dateutil.parser.parse(row['DatePublished'])
    if i <= training_limit:
        arr = [date_pub,row['Name'],row['Text'],row['Label'],row['Summary'],row['Keywords']]
        training_data.append(arr)
    else:
        arr = [date_pub,row['Name'],row['Text'],row['Label'],row['Summary'],row['Keywords']]
        validation_data.append(arr)
    i=i+1 

In [79]:
def getLabels(ip,op):
    for i in range(len(ip)):
        if ip[i][3] == 1.0:
            op.append(1)
        else:
            op.append(0)
    return op

In [80]:
def getTextData(ip,op):
    for i in range(len(ip)):
        op.append(ip[i][2])
    return op

In [87]:
print(len(training_data))
print(len(validation_data))
training_labels = []
validation_labels = []
validation_Features=[]
training_Features =[]

training_labels = getLabels(training_data,training_labels)
validation_labels = getLabels(validation_data,validation_labels)
training_Features = getTextData(training_data,training_Features)
validation_Features = getTextData(training_data,validation_Features)

whole_data = []
whole_labels = []
for index, row in df.iterrows():
    date_pub = dateutil.parser.parse(row['DatePublished'])
    arr = [date_pub,row['Name'],row['Text'],row['Label'],row['Summary'],row['Keywords']]
    if row['Text'] is np.nan:
        whole_data.append("")
    else:
        whole_data.append(row['Text'])
    if row['Label'] == 1.0:
        whole_labels.append(1)
    else:
        whole_labels.append(0)

853
212


In [88]:
from sklearn import cross_validation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif
        
features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(np.array(whole_data), np.array(whole_labels), test_size=0.1, random_state=42)
sample_art = ["SRINAGAR, India (AP) - Anti-India protests and clashes erupted in disputed Kashmir on Saturday after a gunbattle between militants and government forces killed four rebels, police and residents said.According to rights groups, they include at least 269 militants, 158 members of Indian government forces and 156 civilians.Indian troops early Saturday surrounded a southern village in the Pulwama area on a tip that militants were hiding there, leading to an exchange of gunfire, police said.Residents said government forces blasted one civilian house with explosives during the fighting, a common counterinsurgency tactic by Indian troops in Kashmir.Rebels have been fighting Indian control since 1989."]


### text vectorization--go from strings to lists of numbers
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
features_train_transformed = vectorizer.fit_transform(features_train)
features_test_transformed  = vectorizer.transform(features_test)
sample_article = vectorizer.transform(sample_art)

### feature selection, because text is super high dimensional and 
### can be really computationally chewy as a result
selector = SelectPercentile(f_classif, percentile=25)
selector.fit(features_train_transformed, labels_train)
features_train_transformed = selector.transform(features_train_transformed).toarray()
features_test_transformed  = selector.transform(features_test_transformed).toarray()
sample_article = selector.transform(sample_article).toarray()

/Users/dhayanidhigunasekaran/anaconda/lib/python3.5/site-packages/sklearn/feature_selection/univariate_selection.py:108: RuntimeWarning: invalid value encountered in true_divide
  msb = ssbn / float(dfbn)
/Users/dhayanidhigunasekaran/anaconda/lib/python3.5/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


In [89]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(features_train_transformed, labels_train)

GaussianNB(priors=None)

In [90]:
if clf.predict(sample_article) == 1:
    print("possiblity of protests ")
else:
    print("No possibility of protests")

possiblity of protests 
